# Dense vectors (embeddings) in pytorch



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Embedding layer : torch.nn.Embeddings

We've seen that the forward propagation within a neural network may use a "lookup" operation: given an word id (or the id of any other kind of symbol), retrieve a corresponding vector.

This can be handled in pytorch using an instance of the torch.nn.Embeddings class:

https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html

The torch.nn.Embeddings class acts as an "embedding layer" within a neural network, namely it simply corresponds to a matrix of vectors (or "embeddings") for each symbol id in a vocabulary of symbols.

The forward propagation for such a layer corresponds to a lookup operation into a matrix of embeddings.

For instance,
- given an embedding layer E containing embeddings of size 100
- given X a 2x3 matrix containing word ids (imagine two rows, each containing 3 word ids),
- performing the forward operation of E on X will give a 2x3x100 tensor

More generally, **A nn.Embeddings instance, with embeddings of size h, can transform any tensor containing ids into a tensor of vectors for these ids**: if the input shape is (d1, d2, ... dn), the output shape will be (d1, d2, ..., dn, h).

**Moreover, being a subclass of the nn.Module class, nn.Embedding will handle backpropagation : the embeddings can be considered as parameters of a neural network**

### Example with random initialization

In [2]:
# see for instance https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html

VOCAB_SIZE = 10
EMBEDDING_SIZE = 5
my_embedding_layer = nn.Embedding(VOCAB_SIZE, EMBEDDING_SIZE)

# one way among others to randomly initialize the embeddings
initrange = 0.5 / EMBEDDING_SIZE
my_embedding_layer.weight.data.uniform_(-initrange, initrange)
print("10 embeddings of length 5:\n", my_embedding_layer.weight.data)


10 embeddings of length 5:
 tensor([[-0.0941, -0.0672,  0.0284,  0.0420,  0.0454],
        [ 0.0867, -0.0749,  0.0738, -0.0261, -0.0798],
        [ 0.0831, -0.0983,  0.0640, -0.0964, -0.0788],
        [-0.0302, -0.0701, -0.0208,  0.0967, -0.0852],
        [ 0.0118, -0.0939,  0.0502,  0.0758, -0.0580],
        [ 0.0977,  0.0879, -0.0066, -0.0283,  0.0646],
        [ 0.0311, -0.0783, -0.0437, -0.0983, -0.0688],
        [-0.0465,  0.0249,  0.0759,  0.0936, -0.0574],
        [-0.0127,  0.0583, -0.0242, -0.0189, -0.0602],
        [ 0.0510, -0.0633, -0.0002,  0.0160, -0.0923]])


### Initializing with pre-trained embeddings

See the `torch.nn.Embeddings.from_pretrained` method.

Crucially, its `freeze` option will control **whether or not to consider the embeddings as parameters**, i.e. whether or not to update the embeddings in later learning process.


###  Example for an input tensor with a single axis

In [3]:
# Creation of a vector of ids
# NB: we cannot simply use rand, cf. this gives real values between 0 and 1
a = torch.rand(4)
print(a)

# we rather need random integers to play the role of symbol ids (e.g. word ids)
wids = torch.LongTensor(4).random_(0, VOCAB_SIZE-1)
print("wids:", wids)
print(wids.size())

tensor([0.5747, 0.1676, 0.1699, 0.8162])
wids: tensor([8, 8, 5, 1])
torch.Size([4])


In [4]:
# transformation into embeddings
# reminder: for any subclass of nn.Module (hence for nn.Embedding)
#           the forward propagation is done by calling the intance name
wembs = my_embedding_layer(wids)
print("Shape after transformation into embeddings:", wembs.size())
# the embedding of wids[0], i.e. the embedding of the word whose id is 4
print("The embedding for the id %d : "% wids[0], wembs[0])


Shape after transformation into embeddings: torch.Size([4, 5])
The embedding for the id 8 :  tensor([-0.0127,  0.0583, -0.0242, -0.0189, -0.0602],
       grad_fn=<SelectBackward0>)


###  For an input matrix (two axes)

In [5]:
mat = torch.LongTensor(3,2).random_(0, VOCAB_SIZE-1)
print(mat)
print(mat.size())

tensor([[5, 3],
        [5, 4],
        [5, 8]])
torch.Size([3, 2])


In [6]:
# idem, the embedding layer transforms all the ids into dense vectors
# => we obtain a tensor with 3 axes
emb_mat = my_embedding_layer(mat)
print(emb_mat.size())
# the embedding for the id at mat[0,1]
print(mat[0,1])
print(emb_mat[0,1,:])
print(emb_mat)

torch.Size([3, 2, 5])
tensor(3)
tensor([-0.0302, -0.0701, -0.0208,  0.0967, -0.0852], grad_fn=<SliceBackward0>)
tensor([[[ 0.0977,  0.0879, -0.0066, -0.0283,  0.0646],
         [-0.0302, -0.0701, -0.0208,  0.0967, -0.0852]],

        [[ 0.0977,  0.0879, -0.0066, -0.0283,  0.0646],
         [ 0.0118, -0.0939,  0.0502,  0.0758, -0.0580]],

        [[ 0.0977,  0.0879, -0.0066, -0.0283,  0.0646],
         [-0.0127,  0.0583, -0.0242, -0.0189, -0.0602]]],
       grad_fn=<EmbeddingBackward0>)


### TODO1: concatenation and sum of embeddings
Suppose we consider that the `mat` tensor above corresponds to a batch of word id sequences. Namely here that it corresponds to:
- 3 input examples
- each example containing 2 word ids


**1. print a tensor containing for each example the concatenation of the 2 word embeddings
(what is its shape?)**



**2. Print a tensor containing, for each example, the sum of the 2 word embeddings
(what is its shape?)**


In [7]:
# here is what the first row of the resulting tensor should look like
print(emb_mat[0,0,:] + emb_mat[0,1,:])

tensor([ 0.0674,  0.0178, -0.0274,  0.0684, -0.0206], grad_fn=<AddBackward0>)
